# SQL Functions

This notebook covers SnowDuck's function support:
- String functions
- Numeric functions
- Date/time functions
- Conditional functions

In [1]:
import snowflake.connector

from snowduck import start_patch_snowflake

start_patch_snowflake()

## String Functions

In [2]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            UPPER('hello') as upper_val,
            LOWER('WORLD') as lower_val,
            CONCAT('Snow', 'Duck') as concat_val,
            LENGTH('SnowDuck') as len_val
    """)
    row = cursor.fetchone()
    print(f"UPPER('hello') = {row[0]}")
    print(f"LOWER('WORLD') = {row[1]}")
    print(f"CONCAT('Snow', 'Duck') = {row[2]}")
    print(f"LENGTH('SnowDuck') = {row[3]}")

UPPER('hello') = HELLO
LOWER('WORLD') = world
CONCAT('Snow', 'Duck') = SnowDuck
LENGTH('SnowDuck') = 8


In [3]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("SELECT 'Hello' || SPACE(3) || 'World' as spaced")
    print(f"SPACE(3): '{cursor.fetchone()[0]}'")
    
    cursor.execute("SELECT STRTOK('one,two,three', ',', 2) as token")
    print(f"STRTOK('one,two,three', ',', 2) = {cursor.fetchone()[0]}")
    
    cursor.execute("SELECT SPLIT_PART('a-b-c', '-', 2) as part")
    print(f"SPLIT_PART('a-b-c', '-', 2) = {cursor.fetchone()[0]}")

SPACE(3): 'Hello   World'
STRTOK('one,two,three', ',', 2) = two
SPLIT_PART('a-b-c', '-', 2) = b


## Numeric Functions

In [4]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ABS(-42.5) as abs_val,
            CEIL(4.3) as ceil_val,
            FLOOR(4.9) as floor_val,
            ROUND(3.14159, 2) as round_val,
            TRUNCATE(3.99, 1) as trunc_val
    """)
    row = cursor.fetchone()
    print(f"ABS(-42.5) = {row[0]}")
    print(f"CEIL(4.3) = {row[1]}")
    print(f"FLOOR(4.9) = {row[2]}")
    print(f"ROUND(3.14159, 2) = {row[3]}")
    print(f"TRUNCATE(3.99, 1) = {row[4]}")

ABS(-42.5) = 42.5
CEIL(4.3) = 5
FLOOR(4.9) = 4
ROUND(3.14159, 2) = 3.14
TRUNCATE(3.99, 1) = 3.9


In [5]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("SELECT DIV0(10, 2), DIV0(10, 0)")
    row = cursor.fetchone()
    print(f"DIV0(10, 2) = {row[0]}")
    print(f"DIV0(10, 0) = {row[1]}  (returns 0 instead of error)")

DIV0(10, 2) = 5.0
DIV0(10, 0) = 0.0  (returns 0 instead of error)


## Date/Time Functions

In [6]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            CURRENT_DATE() as today,
            DATEADD(day, 7, CURRENT_DATE()) as next_week,
            DATEDIFF(day, '2024-01-01', CURRENT_DATE()) as days_since_jan1
    """)
    row = cursor.fetchone()
    print(f"CURRENT_DATE() = {row[0]}")
    print(f"DATEADD(day, 7, today) = {row[1]}")
    print(f"DATEDIFF(day, '2024-01-01', today) = {row[2]}")

CURRENT_DATE() = 2026-01-22
DATEADD(day, 7, today) = 2026-01-29 00:00:00
DATEDIFF(day, '2024-01-01', today) = 752


In [7]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ADD_MONTHS('2024-01-15', 3) as three_months_later,
            ADD_MONTHS('2024-01-31', 1) as end_of_feb
    """)
    row = cursor.fetchone()
    print(f"ADD_MONTHS('2024-01-15', 3) = {row[0]}")
    print(f"ADD_MONTHS('2024-01-31', 1) = {row[1]}")

ADD_MONTHS('2024-01-15', 3) = 2024-04-15 00:00:00
ADD_MONTHS('2024-01-31', 1) = 2024-02-29 00:00:00


## Conditional Functions

In [8]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            NVL(NULL, 'default') as nvl_null,
            NVL(42, 0) as nvl_value,
            NVL2(100, 'has value', 'no value') as nvl2_result,
            COALESCE(NULL, NULL, 'found!') as coalesce_result
    """)
    row = cursor.fetchone()
    print(f"NVL(NULL, 'default') = {row[0]}")
    print(f"NVL(42, 0) = {row[1]}")
    print(f"NVL2(100, 'has value', 'no value') = {row[2]}")
    print(f"COALESCE(NULL, NULL, 'found!') = {row[3]}")

NVL(NULL, 'default') = default
NVL(42, 0) = 42
NVL2(100, 'has value', 'no value') = has value
COALESCE(NULL, NULL, 'found!') = found!


In [9]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("SELECT IFF(1 > 0, 'yes', 'no')")
    print(f"IFF(1 > 0, 'yes', 'no') = {cursor.fetchone()[0]}")
    
    cursor.execute("SELECT DECODE(2, 1, 'one', 2, 'two', 3, 'three', 'other')")
    print(f"DECODE(2, 1, 'one', 2, 'two', ...) = {cursor.fetchone()[0]}")

IFF(1 > 0, 'yes', 'no') = yes
DECODE(2, 1, 'one', 2, 'two', ...) = two


## Regex Functions

In [10]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            REGEXP_LIKE('hello123', '[a-z]+[0-9]+') as matches,
            REGEXP_SUBSTR('user@example.com', '@[a-z]+') as domain,
            REGEXP_REPLACE('hello world', 'world', 'SnowDuck') as replaced,
            REGEXP_COUNT('banana', 'a') as count_a
    """)
    row = cursor.fetchone()
    print(f"REGEXP_LIKE('hello123', '[a-z]+[0-9]+') = {row[0]}")
    print(f"REGEXP_SUBSTR('user@example.com', '@[a-z]+') = {row[1]}")
    print(f"REGEXP_REPLACE('hello world', 'world', 'SnowDuck') = {row[2]}")
    print(f"REGEXP_COUNT('banana', 'a') = {row[3]}")

REGEXP_LIKE('hello123', '[a-z]+[0-9]+') = True
REGEXP_SUBSTR('user@example.com', '@[a-z]+') = @example
REGEXP_REPLACE('hello world', 'world', 'SnowDuck') = hello SnowDuck
REGEXP_COUNT('banana', 'a') = 3


## Array Functions

In [11]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ARRAY_CONSTRUCT(1, 2, 3) as arr,
            ARRAY_APPEND(ARRAY_CONSTRUCT(1, 2), 3) as appended,
            ARRAY_CAT(ARRAY_CONSTRUCT(1, 2), ARRAY_CONSTRUCT(3, 4)) as concatenated
    """)
    row = cursor.fetchone()
    print(f"ARRAY_CONSTRUCT(1, 2, 3) = {row[0]}")
    print(f"ARRAY_APPEND([1, 2], 3) = {row[1]}")
    print(f"ARRAY_CAT([1, 2], [3, 4]) = {row[2]}")

ARRAY_CONSTRUCT(1, 2, 3) = [1, 2, 3]
ARRAY_APPEND([1, 2], 3) = [1, 2, 3]
ARRAY_CAT([1, 2], [3, 4]) = [1, 2, 3, 4]


In [12]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ARRAY_DISTINCT(ARRAY_CONSTRUCT(1, 2, 2, 3, 3, 3)) as unique_vals,
            ARRAY_INTERSECTION(ARRAY_CONSTRUCT(1, 2, 3), ARRAY_CONSTRUCT(2, 3, 4)) as common,
            ARRAY_EXCEPT(ARRAY_CONSTRUCT(1, 2, 3), ARRAY_CONSTRUCT(2, 3, 4)) as difference
    """)
    row = cursor.fetchone()
    print(f"ARRAY_DISTINCT([1, 2, 2, 3, 3, 3]) = {row[0]}")
    print(f"ARRAY_INTERSECTION([1, 2, 3], [2, 3, 4]) = {row[1]}")
    print(f"ARRAY_EXCEPT([1, 2, 3], [2, 3, 4]) = {row[2]}")

ARRAY_DISTINCT([1, 2, 2, 3, 3, 3]) = [3, 2, 1]
ARRAY_INTERSECTION([1, 2, 3], [2, 3, 4]) = [3, 2]
ARRAY_EXCEPT([1, 2, 3], [2, 3, 4]) = [1]


## Hash & Encoding Functions

In [13]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            SHA1('hello') as sha1_hash,
            SHA2('hello') as sha256_hash,
            MD5('hello') as md5_hash
    """)
    row = cursor.fetchone()
    print(f"SHA1('hello') = {row[0]}")
    print(f"SHA2('hello') = {row[1]}")
    print(f"MD5('hello') = {row[2]}")

SHA1('hello') = aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
SHA2('hello') = 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
MD5('hello') = 5d41402abc4b2a76b9719d911017c592


In [14]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            BASE64_ENCODE('hello') as b64_encoded,
            BASE64_DECODE_STRING('aGVsbG8=') as b64_decoded,
            HEX_ENCODE('hello') as hex_encoded,
            HEX_DECODE_STRING('68656c6c6f') as hex_decoded
    """)
    row = cursor.fetchone()
    print(f"BASE64_ENCODE('hello') = {row[0]}")
    print(f"BASE64_DECODE_STRING('aGVsbG8=') = {row[1]}")
    print(f"HEX_ENCODE('hello') = {row[2]}")
    print(f"HEX_DECODE_STRING('68656c6c6f') = {row[3]}")

BASE64_ENCODE('hello') = aGVsbG8=
BASE64_DECODE_STRING('aGVsbG8=') = hello
HEX_ENCODE('hello') = 68656C6C6F
HEX_DECODE_STRING('68656c6c6f') = hello


## JSON Functions

In [15]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            PARSE_JSON('{"name": "Alice", "age": 30}') as json_obj,
            OBJECT_CONSTRUCT('a', 1, 'b', 2) as obj,
            OBJECT_INSERT(PARSE_JSON('{"a": 1}'), 'b', 2) as inserted
    """)
    row = cursor.fetchone()
    print(f"PARSE_JSON = {row[0]}")
    print(f"OBJECT_CONSTRUCT('a', 1, 'b', 2) = {row[1]}")
    print(f"OBJECT_INSERT({{a: 1}}, 'b', 2) = {row[2]}")

PARSE_JSON = {"name": "Alice", "age": 30}
OBJECT_CONSTRUCT('a', 1, 'b', 2) = {"a":1,"b":2}
OBJECT_INSERT({a: 1}, 'b', 2) = {"a":1,"b":2}
